# Model Selection

In [14]:
import os

import matplotlib.pyplot as plt
import mlflow
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

load_dotenv()

ROOT_DIR = os.path.join(os.getcwd(), '..')
DATA_DIR = os.path.join(ROOT_DIR, 'data')

mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
mlflow.set_experiment('Imbalanced Credit Card Fraud Detection')
mlflow.sklearn.autolog()

## Choosing the metrics

In [15]:
def log_model_results(y_true, y_hat):
    """Calculate, print, and log metrics and artifacts to MLflow."""

    recall = recall_score(y_true, y_hat)
    f1 = f1_score(y_true, y_hat)
    precision = precision_score(y_true, y_hat, zero_division=1)
    accuracy = accuracy_score(y_true, y_hat)

    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Accuracy: {accuracy:.4f}')

    metrics_dict = {
        'test_recall': recall,
        'test_f1_score': f1,
        'test_precision': precision,
        'test_accuracy': accuracy,
    }
    mlflow.log_metrics(metrics_dict)

    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_true, y_hat, labels=[0, 1])
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['No Fraud (P)', 'Fraud (P)'],
        yticklabels=['No Fraud (T)', 'Fraud (T)'],
    )
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.plot()

    confusion_matrix_path = 'confusion_matrix.png'
    plt.savefig(confusion_matrix_path)
    plt.close()

    mlflow.log_artifact(confusion_matrix_path, 'plots')

## Establishing a baseline

In [16]:
df_train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [17]:
with mlflow.start_run(run_name='Baseline Dummy Model'):
    baseline = DummyClassifier(strategy='most_frequent')
    baseline.fit(df_train.drop(columns=['Class']), df_train['Class'])

    y_hat = baseline.predict(df_test.drop(columns=['Class']))

    log_model_results(df_test['Class'], y_hat)

2025/11/01 17:25:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
c:\git_reps\imbalanced_credit_fraud_detection\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Recall: 0.0000
F1 Score: 0.0000
Precision: 1.0000
Accuracy: 0.9983
🏃 View run Baseline Dummy Model at: http://localhost:5000/#/experiments/713154386041343957/runs/6c7cb103add9404294ca8f24e2104625
🧪 View experiment at: http://localhost:5000/#/experiments/713154386041343957


In [18]:
with mlflow.start_run(run_name='Baseline XGBoost Classifier'):
    xgb_baseline = XGBClassifier(eval_metric='aucpr', random_state=42)
    xgb_baseline.fit(df_train.drop(columns=['Class']), df_train['Class'])

    y_hat_xgb = xgb_baseline.predict(df_test.drop(columns=['Class']))
    log_model_results(df_test['Class'], y_hat_xgb)

Recall: 0.7162
F1 Score: 0.7794
Precision: 0.8548
Accuracy: 0.9993
🏃 View run Baseline XGBoost Classifier at: http://localhost:5000/#/experiments/713154386041343957/runs/39bcb4c486dc4745b6f9336c81db1ea6
🧪 View experiment at: http://localhost:5000/#/experiments/713154386041343957


In [19]:
with mlflow.start_run(run_name='Baseline Logistic Regression'):
    lr_baseline = LogisticRegression(
        penalty='l2', random_state=42, n_jobs=-1, max_iter=1000
    )
    lr_baseline.fit(df_train.drop(columns=['Class']), df_train['Class'])

    y_hat_xgb = lr_baseline.predict(df_test.drop(columns=['Class']))
    log_model_results(df_test['Class'], y_hat_xgb)

2025/11/01 17:26:12 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Recall: 0.6216
F1 Score: 0.7244
Precision: 0.8679
Accuracy: 0.9992
🏃 View run Baseline Logistic Regression at: http://localhost:5000/#/experiments/713154386041343957/runs/6144561f26c64003980c30a21a119a3f
🧪 View experiment at: http://localhost:5000/#/experiments/713154386041343957


## Feature Engineering

### Temporal features

In [20]:
df_train['Time'] = pd.to_datetime(df_train['Time'], unit='s')
df_train['Hour'] = df_train['Time'].dt.hour.astype(str)

df_test['Time'] = pd.to_datetime(df_test['Time'], unit='s')
df_test['Hour'] = df_test['Time'].dt.hour.astype(str)

In [21]:
hot_encoder = OneHotEncoder(
    categories='auto',
    drop='first',
    handle_unknown='ignore',
)

hour_encoded_train = hot_encoder.fit_transform(df_train[['Hour']])
hour_encoded_test = hot_encoder.transform(df_test[['Hour']])

hour_encoded_df = pd.DataFrame(
    hour_encoded_train.toarray(),
    columns=hot_encoder.get_feature_names_out(['Hour']),
    index=df_train.index,
)
df_train = pd.concat([df_train, hour_encoded_df], axis=1)
df_train = df_train.drop(columns=['Hour', 'Time'])

hour_encoded_df_test = pd.DataFrame(
    hour_encoded_test.toarray(),
    columns=hot_encoder.get_feature_names_out(['Hour']),
    index=df_test.index,
)
df_test = pd.concat([df_test, hour_encoded_df_test], axis=1)
df_test = df_test.drop(columns=['Hour', 'Time'])

In [22]:
with mlflow.start_run(run_name='Hour XGBoost Classifier'):
    xgb_baseline = XGBClassifier(eval_metric='aucpr', random_state=42)
    xgb_baseline.fit(df_train.drop(columns=['Class']), df_train['Class'])

    y_hat_xgb = xgb_baseline.predict(df_test.drop(columns=['Class']))
    log_model_results(df_test['Class'], y_hat_xgb)

Recall: 0.7162
F1 Score: 0.7970
Precision: 0.8983
Accuracy: 0.9994
🏃 View run Hour XGBoost Classifier at: http://localhost:5000/#/experiments/713154386041343957/runs/18a74581c7e2423eb593fd23ea70cc52
🧪 View experiment at: http://localhost:5000/#/experiments/713154386041343957


In [23]:
with mlflow.start_run(run_name='Hour Logistic Regression'):
    lr_baseline = LogisticRegression(
        penalty='l2', random_state=42, n_jobs=-1, max_iter=1000
    )
    lr_baseline.fit(df_train.drop(columns=['Class']), df_train['Class'])

    y_hat_xgb = lr_baseline.predict(df_test.drop(columns=['Class']))
    log_model_results(df_test['Class'], y_hat_xgb)

2025/11/01 17:27:05 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Recall: 0.6081
F1 Score: 0.7317
Precision: 0.9184
Accuracy: 0.9992
🏃 View run Hour Logistic Regression at: http://localhost:5000/#/experiments/713154386041343957/runs/773cf4738d50445a91eaec8a18bacca8
🧪 View experiment at: http://localhost:5000/#/experiments/713154386041343957
